In [1]:
# import prereq.
from __future__ import absolute_import, division, print_function, unicode_literals
 
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint # used for callback 


In [2]:

X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')

X_c = pd.read_csv('X_c.csv')
Y_c = pd.read_csv('Y_c.csv')

X_t = pd.read_csv('X_t.csv')
Y_t = pd.read_csv('Y_t.csv')

XX_c = X_c.iloc[0:int(X.shape[0]*1/4),:]
YY_c = Y_c.iloc[0:int(X.shape[0]*1/4),:]

X_train = X.iloc[0:int(X.shape[0]*1/4-1),:]
Y_train = Y.iloc[0:int(X.shape[0]*1/4-1),:]

X_pretrain = X.iloc[int(X.shape[0]*1/4):int(X.shape[0]),:]
Y_pretrain = Y.iloc[int(X.shape[0]*1/4):int(X.shape[0]),:]

In [3]:
datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
            zoom_range=0.05,
            width_shift_range=0.05,
            height_shift_range=0.05,
            horizontal_flip=True,
            fill_mode="reflect")

In [4]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# convert to numpy  np.append(a, [[7,8,9]],axis = 0)
data_type =  np.dtype(np.float64) # define datatype

x_Pretrain = np.asarray(X_pretrain.iloc[:,1:].to_numpy(), dtype=data_type)
y_Pretrain = np.asarray(Y_pretrain.iloc[:,1].to_numpy(), dtype=data_type)

x_Train = np.asarray(X.iloc[:,1:].to_numpy(), dtype=data_type)
y_Train = np.asarray(Y.iloc[:,1].to_numpy(), dtype=data_type)

x_t = np.asarray(X_t.iloc[:,1:].to_numpy(), dtype=data_type)
y_t = np.asarray(Y_t.iloc[:,1].to_numpy(), dtype=data_type)
x_c = np.asarray(XX_c.iloc[:,1:].to_numpy(), dtype=data_type)
y_c = np.asarray(YY_c.iloc[:,1].to_numpy(), dtype=data_type)


# clean some memory
X = X_c = X_t = Y = Y_c = Y_t = None

In [6]:
x_train = x_Train.reshape(x_Train.shape[0], img_rows, img_cols, 1)
x_pretrain = x_Pretrain.reshape(x_Pretrain.shape[0], img_rows, img_cols, 1)
x_cross = x_c.reshape(x_c.shape[0], img_rows, img_cols, 1)
x_test = x_t.reshape(x_t.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols,1)

In [7]:
y_train = keras.utils.to_categorical(y_Train, num_classes)
y_pretrain = keras.utils.to_categorical(y_Pretrain, num_classes)
y_cross = keras.utils.to_categorical(y_c, num_classes)
y_test = keras.utils.to_categorical(y_t, num_classes)

In [87]:
# # arhitect the model
# # lastIterationV2
# model = tf.keras.Sequential([

    
# layers.Conv2D(4, kernel_size=(7, 7),
#                  padding='same',
#                  activation='relu',
#                  input_shape=input_shape),  
# layers.Dropout(0.1),
# layers.Conv2D(8, padding='same', kernel_size=(7, 7), activation='relu'),
# layers.Dropout(0.1),

# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.Conv2D(16, padding='same', kernel_size=(7, 7), activation='relu'),
# layers.Dropout(0.1),
# layers.Conv2D(16, padding='same', kernel_size=(7, 7), activation='relu'),
# layers.Dropout(0.1),

# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.Conv2D(32, padding='same', kernel_size=(7, 7), activation='relu'),
# layers.Dropout(0.1),
# layers.Conv2D(32, padding='same', kernel_size=(7, 7), activation='relu'),
# layers.Dropout(0.1),
    
# layers.Flatten(),

# layers.Dense(1024, activation='relu'),#1024
# layers.Dropout(0.1),
    
# layers.Dense(1024, activation='relu'),#1024
# layers.Dropout(0.1),
    
# layers.Dense(1024, activation='relu'),#1024
# layers.Dropout(0.1),
      
# layers.Dense(512, activation='relu'),#1024
# layers.Dropout(0.1),
    
# layers.Dense(100, activation='relu'),#1024
# layers.Dropout(0.1),
        
# layers.Dense(10, activation='softmax')])



# model = tf.keras.Sequential([

    
# layers.Conv2D(28, kernel_size=(3, 3),
#                  padding='same',
#                  activation='relu',
#                  input_shape=input_shape),  
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),
    
# layers.Conv2D(28, padding='same', kernel_size=(3, 3), activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),        
    
# layers.Conv2D(28, padding='same', kernel_size=(3, 3), activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),    

# layers.MaxPooling2D(pool_size=(2, 2)),
    
# layers.Conv2D(56, padding='same', kernel_size=(3, 3), activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),   
    
# layers.Conv2D(56, padding='same', kernel_size=(3, 3), activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),   
    
# layers.Conv2D(56, padding='same', kernel_size=(3, 3), activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),   
    
# layers.MaxPooling2D(pool_size=(2, 2)),

# layers.Flatten(),

# layers.Dense(700, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(700, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(200, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.1),
    
# layers.Dense(60, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.05),
        
# layers.Dense(10, activation='softmax')])


model = tf.keras.Sequential([#retry 97 acc but with lower dropout
layers.Conv2D(4, kernel_size=(4, 4), # 95.7 93.5
                 padding='same',
                 activation='relu',
                 input_shape=input_shape),  
layers.BatchNormalization(momentum=0.99),
layers.Dropout(0.5),
# layers.Dropout(0.5),
# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.MaxPooling2D(pool_size=(2, 2)),
    
# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.Conv2D(16, padding='same',kernel_size=(1, 3), activation='relu'),
# layers.Dropout(0.45),
    
layers.Flatten(),

# layers.Dense(1024, activation='relu'),#1024
# layers.Dropout(0.5),
    
layers.Dense(1024, activation='relu'),#1024 best ----- last 94
layers.BatchNormalization(momentum=0.99),
layers.Dropout(0.4),
    
layers.Dense(200, activation='relu'),#1024
layers.BatchNormalization(momentum=0.99),
layers.Dropout(0.35),  
        
layers.Dense(10, activation='softmax')])

# model = tf.keras.Sequential([ # 99.99 cu 93.5

    
# layers.Conv2D(8, kernel_size=(7, 7),
#                  padding='same',
#                  activation='relu',
#                  input_shape=input_shape),  
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),
    
# layers.Flatten(),

# layers.Dense(2048, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(2048, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(200, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.15),
        
#layers.Dense(10, activation='softmax')])

# model = tf.keras.Sequential([

    
# layers.Conv2D(4, kernel_size=(7, 7),
#                  padding='same',
#                  activation='relu',
#                  input_shape=input_shape),  
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.25),
    
# layers.Flatten(),

# layers.Dense(1024, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(1024, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.2),
    
# layers.Dense(100, activation='relu'),
# layers.BatchNormalization(momentum=0.99),
# layers.Dropout(0.15),

# layers.Dense(10, activation='softmax')])

In [88]:
def step_decay(epoch):
    initial_lrate = 0.00018
    return 1 / np.sqrt(epoch + 1) * initial_lrate
     
def step_decay_train(epoch):
    initial_lrate = 0.00018
    return 0.22 / np.sqrt(epoch + 1) * initial_lrate

In [89]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(beta_1=0.99), # for avg over 100 values
              metrics=['accuracy'])
 
lrate = LearningRateScheduler(step_decay)
mc = ModelCheckpoint('lastIterationV2finnalmodelll_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

callbacks = [es, mc,lrate]

In [90]:
model.fit(x_train,y_train, # pretrain the wheights # pretrain
          epochs=100,
          batch_size=100,
          shuffle=True, 
          verbose=1,
         validation_data=(x_cross, y_cross),callbacks=callbacks)

Train on 277200 samples, validate on 1400 samples
Epoch 1/100
276800/277200 [============================>.] - ETA: 0s - loss: 0.6003 - accuracy: 0.7873
Epoch 00001: val_accuracy improved from -inf to 0.88571, saving model to lastIterationV2finnalmodelll_best.h5
277200/277200 [==============================] - 24s 88us/sample - loss: 0.6002 - accuracy: 0.7873 - val_loss: 0.2847 - val_accuracy: 0.8857
Epoch 2/100
276600/277200 [============================>.] - ETA: 0s - loss: 0.4006 - accuracy: 0.8544
Epoch 00002: val_accuracy improved from 0.88571 to 0.91000, saving model to lastIterationV2finnalmodelll_best.h5
277200/277200 [==============================] - 22s 81us/sample - loss: 0.4005 - accuracy: 0.8545 - val_loss: 0.2370 - val_accuracy: 0.9100
Epoch 3/100
277100/277200 [============================>.] - ETA: 0s - loss: 0.3516 - accuracy: 0.8715
Epoch 00003: val_accuracy improved from 0.91000 to 0.92143, saving model to lastIterationV2finnalmodelll_best.h5
277200/277200 [========

Epoch 27/100
277000/277200 [============================>.] - ETA: 0s - loss: 0.1702 - accuracy: 0.9363
Epoch 00027: val_accuracy did not improve from 0.93571
277200/277200 [==============================] - 22s 79us/sample - loss: 0.1702 - accuracy: 0.9363 - val_loss: 0.1942 - val_accuracy: 0.9314
Epoch 28/100
276600/277200 [============================>.] - ETA: 0s - loss: 0.1692 - accuracy: 0.9368
Epoch 00028: val_accuracy did not improve from 0.93571
277200/277200 [==============================] - 22s 79us/sample - loss: 0.1692 - accuracy: 0.9368 - val_loss: 0.1965 - val_accuracy: 0.9314
Epoch 29/100
276800/277200 [============================>.] - ETA: 0s - loss: 0.1649 - accuracy: 0.9382
Epoch 00029: val_accuracy did not improve from 0.93571
277200/277200 [==============================] - 22s 80us/sample - loss: 0.1649 - accuracy: 0.9382 - val_loss: 0.1987 - val_accuracy: 0.9343
Epoch 30/100
276500/277200 [============================>.] - ETA: 0s - loss: 0.1606 - accuracy: 0.9

In [91]:
model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.22746010380131856, 0.93142855]

In [92]:
from keras.models import load_model

pretrained_model = tf.keras.models.load_model('lastIterationV2finnalmodelll_best.h5')

In [93]:
pretrained_model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2207766910961696, 0.93714285]

In [15]:
pretrained_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(beta_1=0.99), # for avg over 100 values
              metrics=['accuracy'])
 
lrate = LearningRateScheduler(step_decay_train)
mc = ModelCheckpoint('lastIterationV2finnalmodelll_best_finallllllll.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

callbacks = [es, mc,lrate]

In [16]:
pretrained_model.fit(x_train, y_train, batch_size = 100,# realtrain
          epochs=40,
          shuffle=True, 
          verbose=1,
         validation_data=(x_cross, y_cross), callbacks=callbacks)

Train on 277200 samples, validate on 1400 samples
Epoch 1/40
277000/277200 [============================>.] - ETA: 0s - loss: 0.0553 - accuracy: 0.9794
Epoch 00001: val_accuracy improved from -inf to 0.92286, saving model to lastIterationV2finnalmodelll_best_finallllllll.h5
277200/277200 [==============================] - 142s 511us/sample - loss: 0.0553 - accuracy: 0.9794 - val_loss: 0.3152 - val_accuracy: 0.9229
Epoch 2/40
277100/277200 [============================>.] - ETA: 0s - loss: 0.0449 - accuracy: 0.9836
Epoch 00002: val_accuracy improved from 0.92286 to 0.92857, saving model to lastIterationV2finnalmodelll_best_finallllllll.h5
277200/277200 [==============================] - 140s 506us/sample - loss: 0.0449 - accuracy: 0.9836 - val_loss: 0.3472 - val_accuracy: 0.9286
Epoch 3/40
277100/277200 [============================>.] - ETA: 0s - loss: 0.0387 - accuracy: 0.9858
Epoch 00003: val_accuracy did not improve from 0.92857
277200/277200 [==============================] - 138s 

KeyboardInterrupt: 

In [ ]:
# itr = datagen.flow(x_train, y_train, batch_size = x_train.shape[0])
# xxx , yyy = itr.next()

In [ ]:
# xxx.shape

In [ ]:
pretrained_model.save('lastIterationV2finnalmodelll_best_xddddd.h5')

In [ ]:
pretrained_model.evaluate(x_test, y_test)

In [ ]:
#-this will become the gratest